In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.stats import wasserstein_distance
import scipy
from numpy.linalg import norm
import random
from sklearn.metrics import mean_squared_error
import pickle
from datetime import datetime
import time
import os
import nbformat
import pandas as pd

In [2]:
from measure_1d import*
from SFW_L2_1D import*
from SFW_KL_1D import*
from quality_metrics import*
from Homotopy_SFW_KL_1D import*
from Homotopy_SFW_L2_1D import*

In [3]:
np.random.seed(10)

# number of molecules
N_mol = 6

# number of GT generated
NN = 10

# sigma PSF
sigma = 7e-2 

# noise level (Poisson)
noiselevel = 5e3

# tolerance radius for Jaccard index computation
tol = 0.05

# domain 
N_ech = 2048*8 # taglia campionamento
xleft = 0
xright = 1
X = np.linspace(xleft, xright, N_ech)
X_big = np.linspace(xleft-xright, xright, 2*N_ech)
X_certif = np.linspace(xleft, xright, N_ech)

In [4]:
Jacc_L2= np.zeros(NN)
TP_L2= np.zeros(NN); FN_L2= np.zeros(NN); FP_L2= np.zeros(NN)
RMSE_ampl_L2= np.zeros(NN); RMSE_pos_L2= np.zeros(NN)
Jacc_KL= np.zeros(NN)
TP_KL= np.zeros(NN); FN_KL= np.zeros(NN); FP_KL= np.zeros(NN)
RMSE_ampl_KL= np.zeros(NN); RMSE_pos_KL= np.zeros(NN)
lambda_L2 = np.zeros(NN); lambda_KL = np.zeros(NN)
nIter_homotopy_L2 = np.zeros(NN); nIter_homotopy_KL = np.zeros(NN)
sigma_target_L2 = np.zeros(NN); sigma_target_KL = np.zeros(NN)
approx_sigma_target_KL = np.zeros(NN)
CPUtime_L2 = np.zeros(NN)
CPUtime_KL = np.zeros(NN)

# Get the current date and time
current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# Create a new folder for this iteration
folder_name = f'HOMOTOPY_Confronto_10GTrandom_NEW_{current_datetime}'
os.makedirs(folder_name, exist_ok=True)
    
for n in range(NN):
    
    # ground truth
    m_ax0 = rand_gt(N_mol)
    y0 = m_ax0.kernel(X, sigma) #noiseless
    y = m_ax0.acquisition(noiselevel, 0, X, sigma) + 1e-8 #noisy
    
    # background
    bg = 0.01*np.ones(np.shape(y))
    
    # sigma target 
    sigma_target_L2[n] = 1.5*L2TV_cost_funct(m_ax0, X, y, bg, 0, sigma)
    sigma_target_KL[n] = 1.5*KLTV_cost_funct(m_ax0, X, y, bg, 0, sigma)
    approx_sigma_target_KL[n] = len(y)/2*(1/np.sum(y))
    
    # L2-BLASSO rec
    start_time = time.time()
    (m_sfw_L2, nrj_sfw_L2, lmd_seq_L2) = Homotopy_SFW_L2_1D(X, y, bg, sigma, sigma_target_L2[n], nnIter=2*N_mol, nIter=1, 
                                                            c = 15, sliding = 1, pruning = 1, gt = m_ax0)
    Jacc_L2[n], TP_L2[n], FN_L2[n], FP_L2[n], RMSE_ampl_L2[n], RMSE_pos_L2[n] = jaccard_rmse(m_ax0, m_sfw_L2, tol)
    end_time = time.time()
    execution_time_L2 = end_time - start_time
    CPUtime_L2[n] = execution_time_L2
    
    nIter_homotopy_L2[n] = len(lmd_seq_L2)
    if len(lmd_seq_L2)>0:
        lambda_L2[n] = lmd_seq_L2[len(lmd_seq_L2)-1]
            
    # KL-BLASSO rec
    start_time = time.time()
    (m_sfw_KL, nrj_sfw_KL, lmd_seq_KL) = Homotopy_SFW_KULLBACK_1D(X, y, bg, sigma, sigma_target_KL[n], nnIter=2*N_mol, nIter=1, 
                                                                  c = 40, sliding = 1, pruning = 1, gt = m_ax0)
    Jacc_KL[n], TP_KL[n], FN_KL[n], FP_KL[n], RMSE_ampl_KL[n], RMSE_pos_KL[n] = jaccard_rmse(m_ax0, m_sfw_KL, tol)
    end_time = time.time()
    execution_time_KL = end_time - start_time
    CPUtime_KL[n] = execution_time_KL
    
    nIter_homotopy_KL[n] = len(lmd_seq_KL)
    if len(lmd_seq_KL)>0:
        lambda_KL[n] = lmd_seq_KL[len(lmd_seq_KL)-1]
    
    variables_to_save = {
        'm_ax0': m_ax0,
        'y0': y0,
        'y': y,
        'bg': bg,
        'lambda_seq_L2': lmd_seq_L2,
        'm_sfw_L2': m_sfw_L2,
        'nrj_sfw_l2': nrj_sfw_L2,
        'lambda_seq_KL': lmd_seq_KL,
        'm_sfw_KL': m_sfw_KL,
        'nrj_sfw_KL': nrj_sfw_KL
        }

    
    # Define the file path and name for this iteration with date and time
    file_path = os.path.join(folder_name, f'init_{n}.pkl')

    # Save to a file
    with open(file_path, 'wb') as file:
        pickle.dump(variables_to_save, file)

    print(f'Variables for iteration {n} saved successfully in folder {folder_name}.')
    
variables_to_save = {
    'sigma': sigma,
    'noiselevel': noiselevel,
    'tol': tol,
    'm_ax0': m_ax0,
    'y0': y0,
    'bg': bg,
    'Jacc_L2': Jacc_L2,
    'TP_L2': TP_L2,
    'FN_L2': FN_L2,
    'FP_L2': FP_L2,
    'RMSE_ampl_L2': RMSE_ampl_L2,
    'RMSE_pos_L2': RMSE_pos_L2,
    'Jacc_KL': Jacc_KL,
    'TP_KL': TP_KL,
    'FN_KL': FN_KL,
    'FP_KL': FP_KL,
    'RMSE_ampl_KL': RMSE_ampl_KL,
    'RMSE_pos_KL': RMSE_pos_KL,
    'lambda_L2': lambda_L2,
    'lambda_KL': lambda_KL,
    'nIter_homotopy_L2': nIter_homotopy_L2,
    'nIter_homotopy_KL': nIter_homotopy_KL,
    'sigma_target_L2': sigma_target_L2,
    'sigma_target_KL': sigma_target_KL,
    'CPUtime_KL': CPUtime_KL,
    'CPUtime_L2': CPUtime_L2,
    }


# Define the file path and name for this iteration with date and time
file_path = os.path.join(folder_name, f'quality_metrics.pkl')

# Save to a file
with open(file_path, 'wb') as file:
    pickle.dump(variables_to_save, file)
# print(f'Final file with all quality metrics and input variables saved successfully in folder {folder_name}.')


---- Stopping criterion based on the homotopy ----

---- Stopping criterion based on the homotopy ----
Variables for iteration 0 saved successfully in folder HOMOTOPY_Confronto_10GTrandom_NEW_2024-03-28_23-59-04.

---- Stopping criterion based on the homotopy ----

---- Stopping criterion based on the homotopy ----
Variables for iteration 1 saved successfully in folder HOMOTOPY_Confronto_10GTrandom_NEW_2024-03-28_23-59-04.

---- Stopping criterion based on the homotopy ----

---- Stopping criterion based on the homotopy ----
Variables for iteration 2 saved successfully in folder HOMOTOPY_Confronto_10GTrandom_NEW_2024-03-28_23-59-04.

---- Stopping criterion based on the homotopy ----

---- Stopping criterion based on the homotopy ----
Variables for iteration 3 saved successfully in folder HOMOTOPY_Confronto_10GTrandom_NEW_2024-03-28_23-59-04.

---- Stopping criterion based on the homotopy ----

---- Stopping criterion based on the homotopy ----
Variables for iteration 4 saved successf

In [5]:
mean_Jacc_L2 = np.nanmean(Jacc_L2)
mean_TP_L2 = np.nanmean(TP_L2)
mean_FN_L2 = np.nanmean(FN_L2)
mean_FP_L2 = np.nanmean(FP_L2)
mean_RMSE_ampl_L2 = np.nanmean(RMSE_ampl_L2)
mean_RMSE_pos_L2 = np.nanmean(RMSE_pos_L2)
mean_lambda_L2 = np.nanmean(lambda_L2)
mean_nIter_homotopy_L2 = np.nanmean(nIter_homotopy_L2)
mean_sigma_target_L2 = np.nanmean(sigma_target_L2)
mean_Jacc_KL = np.nanmean(Jacc_KL)
mean_TP_KL = np.nanmean(TP_KL)
mean_FN_KL = np.nanmean(FN_KL)
mean_FP_KL = np.nanmean(FP_KL)
mean_RMSE_ampl_KL = np.nanmean(RMSE_ampl_KL)
mean_RMSE_pos_KL = np.nanmean(RMSE_pos_KL)
mean_lambda_KL = np.nanmean(lambda_KL)
mean_nIter_homotopy_KL = np.nanmean(nIter_homotopy_KL)
mean_sigma_target_KL = np.nanmean(sigma_target_KL)
mean_approx_sigma_target_KL = np.nanmean(approx_sigma_target_KL)
mean_CPUtime_KL = np.nanmean(CPUtime_KL)
mean_CPUtime_L2 = np.nanmean(CPUtime_L2)

# Create a DataFrame
data = {
    'Metrics': [
        'mean_Jacc', 'mean_TP', 'mean_FN', 'mean_FP',
        'mean_RMSE_ampl', 'mean_RMSE_pos', 'lambda_estimated',
        'n. homotopy it.', 'mean sigma target', 'approx sigma target KL', 'mean CPU time'
    ],
    'L2-BLASSO': [
        mean_Jacc_L2, mean_TP_L2, mean_FN_L2, mean_FP_L2,
        mean_RMSE_ampl_L2, mean_RMSE_pos_L2, mean_lambda_L2,
        mean_nIter_homotopy_L2, mean_sigma_target_L2, 0, mean_CPUtime_L2
    ],
    'KL-BLASSO': [
        mean_Jacc_KL, mean_TP_KL, mean_FN_KL, mean_FP_KL, 
        mean_RMSE_ampl_KL, mean_RMSE_pos_KL, mean_lambda_KL,
        mean_nIter_homotopy_KL, mean_sigma_target_KL, mean_approx_sigma_target_KL, mean_CPUtime_KL
    ]
}

df = pd.DataFrame(data)
display(df)

,Metrics,L2-BLASSO,KL-BLASSO
0,mean_Jacc,0.740476,0.760119
1,mean_TP,4.500000,4.800000
2,mean_FN,1.500000,1.200000
3,mean_FP,0.100000,0.400000
4,mean_RMSE_ampl,0.411722,0.438368
5,mean_RMSE_pos,0.014930,0.015686
6,lambda_estimated,4.330782,24.542914
7,n. homotopy it.,6.900000,6.400000
8,mean sigma target,1.642378,24.413257
9,approx sigma target KL,0.000000,2.992212
